In [18]:
from openai import api_key
from scipy.stats import cosine

print("OK")

OK


In [19]:
%pwd

'c:\\Coding\\Projects\\Medical-chatbot\\research'

In [20]:
import os
os.chdir("../")

In [21]:
%pwd

'c:\\Coding\\Projects\\Medical-chatbot'

In [22]:
from langchain.document_loaders import PyPDFLoader

In [23]:
# Extract text from PDF files
def extract_text_from_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    return documents

In [24]:
extracted_data=extract_text_from_pdf(r"C:\Coding\Projects\Medical-chatbot\data\Medical_book.pdf")

In [25]:
extracted_data[:2]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]

In [26]:
len(extracted_data)

637

In [27]:
from typing import List # for type hints or function annotations
from langchain.schema import Document

In [28]:
def clean_metadata(docs:List[Document])->List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects containing
    only 'source' in metadata and the original page_content
    :param docs: Original list of Documents with a lot of unwanted metadata information
    :return: Cleaned list of Documents with only necessary metadata and the original page_content
    """
    cleaned_doc: List[Document]=[]
    for doc in docs:
        src=doc.metadata.get("source")
        cleaned_doc.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
        )
    return cleaned_doc

In [29]:
cleaned_document=clean_metadata(extracted_data)

In [30]:
cleaned_document[:2]

[Document(metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content=''),
 Document(metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]

In [31]:
len(cleaned_document)

637

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
# Split the cleaned documents into smaller chunks
def document_into_chunks(cleaned_documents):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts=text_splitter.split_documents(cleaned_documents)
    return texts

In [34]:
chunks=document_into_chunks(cleaned_document)

In [35]:
chunks[:2]

[Document(metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1')]

In [36]:
len(chunks)

5859

In [37]:
from langchain.embeddings import HuggingFaceEmbeddings

In [38]:
# Downloading the embedding model
def download_embeddings():
    """
    Download and return the HuggingFace Embedding model
    :return: HuggingFace embedding model
    """
    model='sentence-transformers/all-MiniLM-L6-v2'
    embeddings=HuggingFaceEmbeddings(
        model_name=model
    )
    return embeddings


In [39]:
embedding_model=download_embeddings()

C:\Users\joabd\AppData\Local\Temp\ipykernel_26452\788224968.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(


In [40]:
embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [41]:
vector=embedding_model.embed_query("Hello world")
vector[:10]

[-0.03447723016142845,
 0.031023241579532623,
 0.006735020782798529,
 0.026109028607606888,
 -0.039362046867609024,
 -0.16030247509479523,
 0.06692402064800262,
 -0.006441468372941017,
 -0.04745054990053177,
 0.01475888304412365]

In [42]:
# Embedding model is a sentence transformer so it converts a sentence into a 384 dimensional vector
len(vector)

384

In [43]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [45]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [46]:
from pinecone import Pinecone
pc=Pinecone(api_key=PINECONE_API_KEY)

In [47]:
pc

In [48]:
from pinecone import ServerlessSpec
index_name="medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384, # Dimensionality of the embedding
        metric="cosine", # Cosine Similarity metric
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index=pc.Index(index_name)

#### **Note:**
- We do not convert the chunks into vectors and store the vectors in the pinecone vector database
- We just give the chunks, the embedding model and the index name, pinecone converts the chunks to the vectors and vice versa using the embedding model we specified.
- <span style="background-color:orange;">When you upload the documents to the pinecone vector database, it does not check if the documents are already present in a given index so there is high possibility of duplication of documents in case you run the code cell twice.</span>

In [49]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embedding_model,
    index_name=index_name
)

In [50]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [51]:
retrieved_docs=retriever.invoke("What is acne?")

In [52]:
retrieved_docs

[Document(id='28abe22a-2bdf-4e9c-a6b9-03d6c02b7579', metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='f1b361b4-82d6-4169-8c88-a5db52670ca6', metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='0b4c6d1c-077f-4f1c-b42f-9dd716396bdf', metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')]

In [53]:
from langchain_openai import ChatOpenAI

ChatModel=ChatOpenAI(model="gpt-4o")

In [54]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [55]:
system_prompt=(
    "You are a medical assistant for question-answering tasks. "
    "Use the following piece of retrieved context to answer "
    "the question. If you do not know the answer, say that you "
    "do not know. Use three sentences at max and keep the answer concise. "
    "\n\n"
    "{context}"
)

In [56]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [57]:
question_answer_chain=create_stuff_documents_chain(ChatModel,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [58]:
response=rag_chain.invoke({"input":"what is Acromegaly and Gigantism ?"})
print(response["answer"])

Acromegaly is a disorder where the abnormal release of a chemical from the pituitary gland in the brain leads to increased growth in bone and soft tissue, along with various disturbances throughout the body. Gigantism shares a similar cause but occurs specifically in children, leading to excessive growth before the growth plates close. Both conditions result from an excess of growth hormone.


In [59]:
response=rag_chain.invoke({"input":"what is acne ?"})
print(response["answer"])

Acne is a skin disorder where the sebaceous glands become inflamed. It affects various areas of the skin, commonly the face, and is known as acne vulgaris.


#### **The following cells shows us the model working without conversational memory:**
- The model is not able to correctly answer questions related to the `treatment of acne`, as this question was asked indirectly not explicitly as the other questions asked before.

In [60]:
response=rag_chain.invoke({"input":"what is the best treatment for it ?"})
print(response["answer"])

The best treatment often depends on the condition being addressed. Conventional treatments might include surgery for severe deformities, while alternative treatments can involve herbal remedies like white willow and yarrow, acupuncture for pain management, and homeopathic remedies for inflammation. It is important to consult with a healthcare provider to tailor the treatment to individual needs.


In [61]:
response=rag_chain.invoke({"input":"what are we talking about right now ?"})
print(response["answer"])

The context provided covers "bites and stings" and "bed-wetting."


**Creating a LLM with Conversational Memory:**
- we create a history aware retriever that makes a retriever understand conversation context.
- we use message placeholder for chat history in prompts. 
- we keep a track of human and AI messages, these are structured message types for conversation history.

In [62]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [63]:
contextualize_system_prompt="""Given a chat history and the latest user question which might reference context
in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer
the question, just reformulate it if needed otherwise return it as is."""

In [64]:
contextualize_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [65]:
# create history aware retriever
history_aware_retriever=create_history_aware_retriever(
    llm=ChatModel,retriever=retriever,prompt=contextualize_prompt
)

In [66]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000022ABD6E4470>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai

In [81]:
# create new document chain with history
qa_system_prompt = """You are a medical assistant for question-answering tasks.
    Use the following piece of retrieved context to answer
    the question. If you do not know the answer, say that you
    do not know. Use a maximum of three sentences and keep the answer concise.
    
    Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
])

qa_chain = create_stuff_documents_chain(llm=ChatModel,prompt=qa_prompt)

# creating conversational rag chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever,
    qa_chain
)
print("Conversational RAG chain created!")

Conversational RAG chain created!


In [82]:
# First question
chat_history=[]
result = conversational_rag_chain.invoke({
    "chat_history":chat_history,
    "input": "What is acne ?"
})
print(f"Q: What is acne?")
print(f"A: {result['answer']}")

Q: What is acne?
A: Acne is a skin disorder characterized by the inflammation of the sebaceous glands.


In [83]:
# Based on the first question we add the human message and AI message to the chat history
chat_history.extend([
    HumanMessage(content = "What is acne ?"),
    AIMessage(content=result['answer'])
])

In [84]:
chat_history

[HumanMessage(content='What is acne ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Acne is a skin disorder characterized by the inflammation of the sebaceous glands.', additional_kwargs={}, response_metadata={})]

In [85]:
# Follow up question
result1 = conversational_rag_chain.invoke({
    "chat_history":chat_history,
    "input": "what is the best treatment for it ?" # refers to acne from the previous question
})
print(f"Q: what is the best treatment for it ?")
print(f"A: {result1['answer']}")

Q: what is the best treatment for it ?
A: The best treatment for acne depends on its severity. Mild noninflammatory acne is typically treated with topical medications such as tretinoin, benzoyl peroxide, adapalene, or salicylic acid, sometimes combined with topical antibiotics if inflammation is present. For severe cases, treatment varies, and a healthcare professional should be consulted for personalized recommendations.


In [86]:
result1

{'chat_history': [HumanMessage(content='What is acne ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Acne is a skin disorder characterized by the inflammation of the sebaceous glands.', additional_kwargs={}, response_metadata={})],
 'input': 'what is the best treatment for it ?',
 'context': [Document(id='cd0bea9f-6174-4331-b9e9-33de4020b04e', metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='depends upon whether the acne is mild, moderate, or\nsevere.\nDrugs\nTOPICAL DRUGS. Treatment for mild noninflamma-\ntory acne consists of reducing the formation of new\ncomedones with topical tretinoin, benzoyl peroxide, ada-\npalene, or salicylic acid. Tretinoin is especially effective\nbecause it increases turnover (death and replacement) of\nskin cells. When complicated by inflammation, topical\nantibiotics may be added to the treatment regimen.\nImprovement is usually seen in two to four weeks.'),
  Document(id='45e8a28b-

In [87]:
chat_history.extend([
    HumanMessage(content = "what is the best treatment for it ?"),
    AIMessage(content=result1['answer'])
])

In [88]:
chat_history

[HumanMessage(content='What is acne ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Acne is a skin disorder characterized by the inflammation of the sebaceous glands.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the best treatment for it ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The best treatment for acne depends on its severity. Mild noninflammatory acne is typically treated with topical medications such as tretinoin, benzoyl peroxide, adapalene, or salicylic acid, sometimes combined with topical antibiotics if inflammation is present. For severe cases, treatment varies, and a healthcare professional should be consulted for personalized recommendations.', additional_kwargs={}, response_metadata={})]

In [89]:
# Follow up question
result2 = conversational_rag_chain.invoke({
    "chat_history":chat_history,
    "input": "What are we talking about ?" # refers to acne from the previous question
})
print(f"Q: What are we talking about ?")
print(f"A: {result2['answer']}")

Q: What are we talking about ?
A: We are discussing acne and its treatments.
